In [ ]:
# %%
# importation des bibliothèques
import requests
import pandas as pd
import sqlite3, sqlalchemy
from tqdm.notebook import trange, tqdm
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Float


# liste des urls pour chaque fichier à telecharger
urls = ['https://datasets.imdbws.com/name.basics.tsv.gz',
       'https://datasets.imdbws.com/title.basics.tsv.gz', 
       'https://datasets.imdbws.com/title.episode.tsv.gz', 
       'https://datasets.imdbws.com/title.principals.tsv.gz', 
       'https://datasets.imdbws.com/title.ratings.tsv.gz']

# # telechargement des fichiers
# for url in urls:

#     target_path = str(url[28:])
#     response = requests.get(url, stream=True)  
   
#     if response.status_code == 200:
#         with open(target_path, 'wb') as f:
#             f.write(response.raw.read())
            
#     for i in trange(1, desc='Statut'):
#         print('Fichier téléchargé :', target_path)

# %%
# création de la base de données crew.db
engine = create_engine('sqlite:///crew.db', echo=False)
meta = MetaData()

import csv

def read_tsv(path):
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        headers = next(reader)

        for row in reader:
            yield {
                h: (None if v == r"\N" else v)
                for h, v in zip(headers, row)
            }




import gzip
import csv

# with gzip.open("title.principals.tsv.gz", "rt", encoding="utf-8") as f:
#     reader = csv.DictReader(f, delimiter="\t")

#     for i, row in enumerate(reader):
#         print(row)
#         if i == 4:
            # break


#inspect data in file

with gzip.open("data/raw/title.principals.tsv.gz", "rt", encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter="\t")

    for i, row in enumerate(reader):
        print(row)
        if i == 4: 
            break



{'tconst': 'tt0000001', 'ordering': '1', 'nconst': 'nm1588970', 'category': 'self', 'job': '\\N', 'characters': '["Self"]'}
{'tconst': 'tt0000001', 'ordering': '2', 'nconst': 'nm0005690', 'category': 'director', 'job': '\\N', 'characters': '\\N'}
{'tconst': 'tt0000001', 'ordering': '3', 'nconst': 'nm0005690', 'category': 'producer', 'job': 'producer', 'characters': '\\N'}
{'tconst': 'tt0000001', 'ordering': '4', 'nconst': 'nm0374658', 'category': 'cinematographer', 'job': 'director of photography', 'characters': '\\N'}
{'tconst': 'tt0000002', 'ordering': '1', 'nconst': 'nm0721526', 'category': 'director', 'job': '\\N', 'characters': '\\N'}


In [ ]:
# %%
################################################## title.principals.tsv.gz ##################################################
# chargement des données
df_title_principals = pd.read_csv('data/raw/title.principals.tsv.gz', compression='gzip', header=0, sep='\t')

# suppresion d'une colonne
df_title_principals.pop('ordering')
# nettoyage des données

# transformation des données du dataframe en liste de tuple
values = df_title_principals.to_records(index=False).tolist()

# création de la table ratings
crew = Table(
    'crew', meta, 
    Column('title_id', String(20), primary_key=False), 
    Column('person_id', String(20)), 
    Column('category', String(20)), 
    Column('job', String(200)), 
    Column('characters', String(200)) )
meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=crew.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
            
# drop des data ratings
df_title_principals = []
values = []

In [11]:
engineIMDB = create_engine('sqlite:///crew.db')
connIMDB = engineIMDB.connect()

result = connIMDB.execute("SELECT title_id,person_id,job,category,characters" 
                          " FROM crew" 
                          " WHERE category='actor'" 
                          " LIMIT 10")
result.fetchall()

[('tt0000005', 'nm0443482', '\\N', 'actor', '["Blacksmith"]'),
 ('tt0000005', 'nm0653042', '\\N', 'actor', '["Assistant"]'),
 ('tt0000007', 'nm0179163', '\\N', 'actor', '\\N'),
 ('tt0000007', 'nm0183947', '\\N', 'actor', '\\N'),
 ('tt0000008', 'nm0653028', '\\N', 'actor', '["Sneezing Man"]'),
 ('tt0000009', 'nm0183823', '\\N', 'actor', '["Mr. Hamilton"]'),
 ('tt0000009', 'nm1309758', '\\N', 'actor', '["Chauncey Depew - the Director of the New York Central Railroad"]'),
 ('tt0000011', 'nm3692297', '\\N', 'actor', '["Acrobats"]'),
 ('tt0000014', 'nm0166380', '\\N', 'actor', '["The Gardener"]'),
 ('tt0000014', 'nm0244989', '\\N', 'actor', '["The Boy"]')]

In [13]:
import sqlite3


def sqlite_db_to_mysql_sql(
    sqlite_db_path: str,
    mysql_sql_path: str,
    default_varchar_len: int = 255,
):
    """
    Convert a SQLite .db file into a MySQL-compatible .sql file.
    Cleans IMDb-style '\\N' → NULL.
    """

    def map_type(sqlite_type: str) -> str:
        if not sqlite_type:
            return f"VARCHAR({default_varchar_len})"

        t = sqlite_type.upper()

        if "INT" in t:
            return "INT"
        if "REAL" in t or "FLOAT" in t or "DOUBLE" in t:
            return "DOUBLE"
        if "BLOB" in t:
            return "BLOB"
        if "CHAR" in t or "CLOB" in t or "TEXT" in t:
            return f"VARCHAR({default_varchar_len})"
        if "NUMERIC" in t:
            return "DECIMAL(18,6)"

        return f"VARCHAR({default_varchar_len})"

    def clean_value(v):
        if v is None:
            return "NULL"
        if isinstance(v, (int, float)):
            return str(v)
        if v == r"\N":
            return "NULL"
        return "'" + str(v).replace("\\", "\\\\").replace("'", "''") + "'"

    conn = sqlite3.connect(sqlite_db_path)
    cursor = conn.cursor()

    with open(mysql_sql_path, "w", encoding="utf-8") as f:
        f.write("SET foreign_key_checks = 0;\n\n")

        # Get user tables
        cursor.execute("""
            SELECT name FROM sqlite_master
            WHERE type='table' AND name NOT LIKE 'sqlite_%'
        """)
        tables = [t[0] for t in cursor.fetchall()]

        for table in tables:
            cursor.execute(f"PRAGMA table_info(`{table}`)")
            cols = cursor.fetchall()

            col_defs = []
            pk_cols = []

            for _, name, col_type, notnull, _, pk in cols:
                mysql_type = map_type(col_type)
                col_def = f"`{name}` {mysql_type}"

                if notnull:
                    col_def += " NOT NULL"

                col_defs.append(col_def)

                if pk:
                    pk_cols.append(name)

            if pk_cols:
                col_defs.append(
                    "PRIMARY KEY (" +
                    ", ".join(f"`{c}`" for c in pk_cols) +
                    ")"
                )

            f.write(f"DROP TABLE IF EXISTS `{table}`;\n")
            f.write(f"CREATE TABLE `{table}` (\n  ")
            f.write(",\n  ".join(col_defs))
            f.write("\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;\n\n")

            cursor.execute(f"SELECT * FROM `{table}`")
            col_names = [d[0] for d in cursor.description]

            for row in cursor:
                values = ", ".join(clean_value(v) for v in row)
                f.write(
                    f"INSERT INTO `{table}` "
                    f"({', '.join(f'`{c}`' for c in col_names)}) "
                    f"VALUES ({values});\n"
                )

            f.write("\n")

        f.write("SET foreign_key_checks = 1;\n")

    conn.close()
    print(f"✅ MySQL dump written to {mysql_sql_path}")



In [ ]:
sqlite_db_to_mysql_sql(
    sqlite_db_path="newIMDB.db",
    mysql_sql_path="testnewIMDB.sql"
)

OSError: [Errno 28] No space left on device